# Energy Reconstruction Using CNN - Both Charges and Cos(Zenith)

In [ ]:
import numpy as np
import os
from csv import writer
from data_tools import dataPrep, getRecoFilter, load_preprocessed
from keras.callbacks import CSVLogger, EarlyStopping
from keras.layers import Concatenate, Conv2D, Dense, Flatten, Input
from keras.models import Model

## Model Design

In [ ]:
# Baseline data prep
prep = {'clc':True, 'sta5':False, 'q':None, 't':None, 't_shift':True, 'normed':True, 'reco':'plane', 'cosz':False}

# Set the number of epochs the model should run for 
numepochs = 100

# Name for model
name = '' + '_%se' % numepochs

# File directory to folder that holds models
modelPrefix = os.getcwd()+'/models'

# Add identifying number to name
i = 0
# Saves the h5 file of the model in a folder named models 
while(os.path.exists('%s/%s.h5' % (modelPrefix, name+str(i)))): 
    i += 1
name += str(i)
print(name)

# File directory to folder that holds simulation data 
simPrefix = os.getcwd()+'/simdata'

# Sim data to reconstruct (dir produces theta & phi, make sure to transpose)
sim = 'energy'

In [ ]:
# Load simulation data from files for training
x, y = load_preprocessed(simPrefix, 'train')

# Prepare simulation data
x_i, idx = dataPrep(x, y, **prep)

In [ ]:
# Filter NaNs from reconstruction data
if prep['reco'] != None:
    NaNfilter = getRecoFilter(prep['reco'], y)

    for i, _ in enumerate(x_i):
        x_i[i] = x_i[i][NaNfilter]

    for key in y.keys():
        y[key] = y[key][NaNfilter]
    
    loss = (len(NaNfilter)-sum(NaNfilter)) / len(NaNfilter) * 100
    print("Percentage of events with a NaN in reconstruction: %.02f" % loss)

In [ ]:
# Create model using functional API for multiple inputs

# Charge is always included as a parameter in the model - highest correlation with energy of all inputs
# Charge input layer 
charge_input = Input(shape=(10,10,idx), name='charge')
# Starts off with three convolutional layers, each one has half the neurons of the previous
q_conv1_layer = Conv2D(64, kernel_size=3, padding='same', activation='relu')(charge_input)
q_conv2_layer = Conv2D(32, kernel_size=3, padding='same', activation='relu')(q_conv1_layer)
q_conv3_layer = Conv2D(16, kernel_size=3, padding='same', activation="relu")(q_conv2_layer)
# Layer is Flattened before Concatenated
q_flat_layer = Flatten()(q_conv3_layer)

# Time has been found to provide information comparable to Zenith when given to more advanced CNNs
if prep['t'] != False: # Whether time has been included as a parameter in the model
    # Time input layer 
    time_input = Input(shape=(10,10,x_i[0].shape[-1]-idx), name='time')
    # Starts off with three convolutional layers, each one has half the neurons of the previous
    t_conv1_layer = Conv2D(64, kernel_size=3, padding='same', activation='relu')(time_input)
    t_conv2_layer = Conv2D(32, kernel_size=3, padding='same', activation='relu')(t_conv1_layer)
    t_conv3_layer = Conv2D(16, kernel_size=3, padding='same', activation="relu")(t_conv2_layer)
    # Layer is Flattened before Concatenated
    t_flat_layer = Flatten()(t_conv3_layer)

# Flat layers are Concatenated before being passed into Dense layers
if prep['reco'] == None: # Whether Zenith has been included as a parameter in the model
    if prep['t'] == False: # Whether Time has been included as a parameter in the model
        raise Exception('Why are you training the model on charge alone? It is not worth it.')
    else:
        concat_layer = Concatenate()([q_flat_layer, t_flat_layer])
else:
    # Zenith input layer
    zenith_input = Input(shape=(1), name='zenith')
    if prep['t'] == False: # Whether Time has been included as a parameter in the model
        concat_layer = Concatenate()([q_flat_layer, zenith_input])
    else:
        concat_layer = Concatenate()([q_flat_layer, t_flat_layer, zenith_input])
    
# The Concatenated layers run through three Dense layers
dense1_layer = Dense(256, activation='relu')(concat_layer)
dense2_layer = Dense(256, activation='relu')(dense1_layer)
dense3_layer = Dense(256, activation="relu")(dense2_layer)

# This last Dense layer is the output of the model
output = Dense(1)(dense3_layer)

# Prepare model for compilation
if prep['reco'] == None: # Whether Zenith has been included as a parameter in the model
    model = Model(inputs=[charge_input, time_input], outputs=output, name=name)
else:
    if prep['t'] == False: # Whether Time has been included as a parameter in the model
        model = Model(inputs=[charge_input, zenith_input], outputs=output, name=name)
    else:
        model = Model(inputs=[charge_input, time_input, zenith_input], outputs=output, name=name)
    
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae','mse'])
model.summary()

In [ ]:
# Training

csv_logger = CSVLogger('%s/%s' % (modelPrefix, name))
# Earlystoping stops the model from training when it starts to overfit to the data
# The main parameter we change is the patience 
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min', baseline=None, restore_best_weights=False) 
callbacks = [early_stop, csv_logger]

if prep['reco'] == None: # Whether Zenith has been included as a parameter in the model
    history = model.fit({'charge':x_i[...,:idx], 'time':x_i[...,idx:]}, y=y[sim], epochs=numepochs, validation_split=0.15, callbacks=callbacks)
else:
    history = model.fit({'charge':x_i[0][...,:idx], 'time':x_i[0][...,idx:], 'zenith':x_i[1].reshape(-1,1)}, y=y[sim], epochs=numepochs, validation_split=0.15, callbacks=callbacks)

In [ ]:
# Save the model results as a .npy and .h5 file
model.save('%s/%s.h5' % (modelPrefix, name))
np.save('%s/%s.npy' % (modelPrefix, name), prep)

# Open a .csv file and write the results of the last epoch
num_epoch = len(history.history['loss'])
val_loss = np.min(history.history['val_loss'])
index = history.history['val_loss'].index(val_loss)
loss = history.history['loss'][index]
new_row = [name, num_epoch, loss, val_loss]
with open('models/results.csv', 'a') as f:
    csv_writer = writer(f)
    csv_writer.writerow(new_row)
f.close() 